In [ ]:
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\ABK-Temp\Dev Data.csv')
data['CIMIS Air Temp'] = data['CIMIS Air Temp'].values + 273.15
data['Front Side'] = data['Front Side'].values + 273.15
data['Bottom Side'] = data['Bottom Side'].values + 273.15
data['Upper Side'] = data['Upper Side'].values + 273.15
data['Temp'] = data['Temp'].values + 273.15
data['Sol Rad'] = data['Sol Rad'].values
data.tail()

In [ ]:
##for side walls
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W_S, x_W_S, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W_S + x_W_S*k_W_S*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta, k_W_S, x_W_S, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta, k_W_S, x_W_S, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_W_S = ps['k_W_S'].value #lumped thermal conductivity for side wall
    x_W_S = ps['x_W_S'].value #lumped heat transfer coefficient = x_W_S*k_W_S
    k_M = ps['k_M'].value # lumped thermal conductivity for hive core
    model = []
    for i,j in data_group:
        environment_temp = j['CIMIS Air Temp'].values #environment data
        solar_radiation = j['Sol Rad'].values #solar radiation
        hive_temp = j['Temp'].values #hive core temperature
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp,
                                 k_W_S, x_W_S, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Front Side']).ravel()

params = Parameters()
params.add('k_W_S', value = 1, min = 0, max = 1)
params.add('x_W_S', value = 0.005, min = 0.000001, max = np.inf)
params.add('k_M', value = 0.1, min = 0, max = 1)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Front Side'] + result.residual[:len(data['Front Side'])].reshape(data['Front Side'].shape)
print('k_W_S', result.params['k_W_S'].value)
print('x_W_S', result.params['x_W_S'].value)
print('k_M', result.params['k_M'].value)
k_W_S = result.params['k_W_S'].value
x_W_S = result.params['x_W_S'].value
k_M = result.params['k_M'].value

In [ ]:
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
plt.plot(data['Front Side'], label = 'Empty Box Actual', color = 'red')
plt.plot(sig_min_front, label = 'Empty Box Reconstructed', color = 'blue')
plt.legend()
plt.show()

In [ ]:
##for top wall
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W_t, x_W_t):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W_t + x_W_t*k_W_t*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_W_t, x_W_t):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_W_t, x_W_t), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_W_t = ps['k_W_t'].value #lumped thermal conductivity for top wall
    x_W_t = ps['x_W_t'].value #lumped heat transfer coefficient = x_W_t*k_W_t
    #print(k_W_S, c_W_S)
    model = []
    for i,j in data_group:
        environment_temp = j['CIMIS Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_W_t, x_W_t)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Upper Side']).ravel()

params = Parameters()
params.add('k_W_t', value = 1, min = 0, max = 1)
params.add('x_W_t', value = 0.005, min = 0.0001, max = np.inf)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_upper = data['Upper Side'] + result.residual[:len(data['Upper Side'])].reshape(data['Upper Side'].shape)
print('k_W_t', result.params['k_W_t'].value)
print('x_W_t', result.params['x_W_t'].value)
k_W_t = result.params['k_W_t'].value
x_W_t = result.params['x_W_t'].value

In [ ]:
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
plt.plot(data['Upper Side'], label = 'Empty Box Actual', color = 'red')
plt.plot(sig_min_front, label = 'Empty Box Reconstructed', color = 'blue')
plt.legend()
plt.show()

In [ ]:
##for bottom walls
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, x_W_B):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W_B + x_W_B*k_W_B*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta, x_W_B):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta, x_W_B), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    x_W_B = ps['x_W_B'].value #lumped heat transfer coefficient = x_W_B*k_W_B
    model = []
    for i,j in data_group:
        environment_temp = j['CIMIS Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values 
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp,
                                 x_W_B)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Bottom Side']).ravel()

params = Parameters()
k_W_B = k_W_S #same as side walls
params.add('x_W_B', value = 0.001, min = 0.0001, max = 0.01)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 100)
#result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_bottom = data['Bottom Side'] + result.residual[:len(data['Bottom Side'])].reshape(data['Bottom Side'].shape)

print('x_W_B', result.params['x_W_B'].value)
x_W_B = result.params['x_W_B'].value

In [ ]:
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
plt.plot(data['Bottom Side'], label = 'Empty Box Actual', color = 'red')
plt.plot(sig_min_front, label = 'Empty Box Reconstructed', color = 'blue')
plt.legend()
plt.show()

In [ ]:
fig, (ax1, ax3, ax2) = plt.subplots(1, 3, sharey=True,figsize=(15, 4), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),12)]
legend_properties = {'size': 14,'weight':'bold'}
ax1 = plt.subplot(131)
ax1.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
ax1.plot(data['Front Side'], label = 'Actual Wall Temperature', color = 'red')
ax1.plot(sig_min_front, label = 'Reconstucted Wall Temperature', color = 'blue')
ax1.set_ylabel('Hourly Temperature (K)', fontsize=14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),12))
ax1.set_xticklabels(y,rotation=90)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
ax1.set_ylim([280,320])
#ax1.legend(loc='upper left', prop=legend_properties)
ax1.text(-6.5,282,  r'$k_W = 0.93, k_M = 0.14,$ $c_1 = 0.0033$', bbox = {'facecolor': 'white', 'pad' :3}, fontsize = 14, fontweight = 'black')
ax2 = plt.subplot(133)
ax2.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
ax2.plot(data['Bottom Side'], label = 'Average Bottom Wall Temperature', color = 'red')
ax2.plot(sig_min_bottom, label = ' Bottom Wall Temperature Reconstructed', color = 'blue')
ax2.set_xticks(range(0,len(data['Date'].values),12))
ax2.set_xticklabels(y,rotation=90)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=10)
#ax2.set_yticks([])
ax2.text(50,282,  r'$c_3 = 0.0029$', bbox = {'facecolor': 'white', 'pad' :3}, fontsize = 14, fontweight = 'black')
#ax2.legend(loc='upper left', prop=legend_properties)
ax3 = plt.subplot(132)
ax3.plot(data['CIMIS Air Temp'], label = 'Environment Temperature', color = 'green')
ax3.plot(data['Upper Side'], label = 'Average Upper Wall Temperature', color = 'red')
ax3.plot(sig_min_upper, label = ' Upper Wall Temperature Reconstructed', color = 'blue')
ax3.set_xlabel('Date & Time', size = 14, fontweight='bold')
ax3.set_xticks(range(0,len(data['Date'].values),12))
ax3.set_xticklabels(y,rotation=90)
ax3.yaxis.set_tick_params(labelsize=12)
ax3.xaxis.set_tick_params(labelsize=10)
#ax3.set_yticks([])
ax3.text(40,282,  r'$k_{W_t} = 0.7, c_2 = 0.0021$', bbox = {'facecolor': 'white', 'pad' :3}, fontsize = 14, fontweight = 'black')
#ax3.legend(loc='upper left', prop=legend_properties)
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, prop = legend_properties, loc='upper center', bbox_to_anchor=(0.51, 1.03),
          fancybox=True, shadow=True, ncol=3)
ax1.text(45,330,  'Side Wall', bbox = {'facecolor': 'white', 'pad' :5}, fontsize = 18, fontweight = 'black')
ax2.text(45,330,  'Bottom Wall', bbox = {'facecolor': 'white', 'pad' :5}, fontsize = 18, fontweight = 'black')
ax3.text(45,330,  'Top Wall', bbox = {'facecolor': 'white', 'pad' :5}, fontsize = 18, fontweight = 'black')
plt.show()

In [ ]:
#k_M (lumped thermal conductivity is different for each hive) estimation
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv') #For control hive
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_no_treat.csv') #For treated hive -> only non-treated days
data = data[:7*24] #first seven days of data

In [ ]:
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W_t + x_W_t*k_W_t*solar_rad - 5*k_M*(theta_W - theta) + 4*k_W_S*(theta_E - theta_W) + 4*x_W_S*k_W_S*solar_rad
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_M = ps['k_M'].value #lumped thermal conductivity hive core
    model = []
    for i,j in data_group:
        environment_temp = j['Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Temp_Easy']).ravel()

params = Parameters()
params.add('k_M', value = 0.5, min = 0, max = np.inf)
k_W_t =  0.6699726312530309 #lumped thermal conductivity top wall
x_W_t = 0.0021755653867763236 #lumped heat transfer coeff top wall 
k_W_S= 0.93 #lumped thermal conductivity side wall
x_W_S = 0.0033 #lumped heat transfer coeff side wall
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Temp_Easy'] + result.residual[:len(data['Temp_Easy'])].reshape(data['Temp_Easy'].shape)
print('k_M', result.params['k_M'].value)

In [ ]:
#test the value for entire dataset
import scipy.integrate as scint
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv')
def bee_eq(y, t, theta_E, solar_rad, theta):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W_t + x_W_t*k_W_t*solar_rad - 5*k_M*(theta_W - theta) + 4*k_W_S*(theta_E - theta_W) + 4*x_W_S*k_W_S*solar_rad
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

k_M = 1.55 #lumped thermal conductivity hive core
k_W_t =  0.6699726312530309 #lumped thermal conductivity top wall
x_W_t = 0.0021755653867763236 #lumped heat transfer coeff top wall 
k_W_S= 0.93 #lumped thermal conductivity side wall
x_W_S = 0.0033 #lumped heat transfer coeff side wall

t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
a = [1]*len(data)
temp_box_recons = run_bee_eq(t,a, data['Temp_Box']+273.15, data['Sol Rad'], data['Temp']+273.15)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp_Easy'], color = 'blue', label = 'Recorded Peripheral Temperature')
plt.plot(temp_box_recons-273.15, color = 'red', label = 'Reconstructed Peripheral Temperature (no ice)')
plt.legend()
#plt.plot(data['Temp_Box'], color = 'yellow')

In [ ]:
#Discard 2022 data, 
#consider all walls have same parameters and 
#estimate kW (=0.62), xW (=0.012), kM (=1) 
#considering peripheral temperature is affected by all walls and hive core expect bottom wall

In [ ]:
#k_W, k_M, x_W_t, x_W_S estimation
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv') #For control hive
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_no_treat.csv') #For treated hive -> only non-treated days
data = data[:10*24] #first seven days of data

In [ ]:
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W, x_W, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W*5 + 5*x_W*k_W*solar_rad - 5*k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_W, x_W, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_W, x_W, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_M = ps['k_M'].value #lumped thermal conductivity hive core
    x_W = ps['x_W'].value
    k_W = ps['k_W'].value
    model = []
    for i,j in data_group:
        environment_temp = j['Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_W, x_W, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Temp_Easy']).ravel()

params = Parameters()
params.add('k_M', value = 1, min = 0, max = 1)
params.add('x_W', value = 0.005, min = 0.000001, max = np.inf)
params.add('k_W', value = 1, min = 0.000001, max = 1)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Temp_Easy'] + result.residual[:len(data['Temp_Easy'])].reshape(data['Temp_Easy'].shape)
print('k_M', result.params['k_M'].value)
print('x_W', result.params['x_W'].value)
print('k_W', result.params['k_W'].value)

In [ ]:
#test the value for entire dataset
import scipy.integrate as scint
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv')
def bee_eq(y, t, theta_E, solar_rad, theta):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W*5 + 5*x_W*k_W*solar_rad - 5*k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

k_M = result.params['k_M'].value #lumped thermal conductivity hive core
k_W = result.params['k_W'].value #lumped thermal conductivity top wall
x_W = result.params['x_W'].value #0.012482890342958908 #lumped heat transfer coeff top wall 

t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
a = [1]*len(data)
temp_box_recons = run_bee_eq(t,a, data['Temp_Box']+273.15, data['Sol Rad'], data['Temp']+273.15)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp_Easy'], color = 'blue', label = 'Recorded Peripheral Temperature')
plt.plot(temp_box_recons-273.15, color = 'red', label = 'Reconstructed Peripheral Temperature (no ice)')
plt.legend()

In [ ]:
#Discard 2022 data, 
#consider all walls have same parameters and 
#estimate kW (=0.62), xW (=0.012), kM (=1) 
#considering peripheral temperature as top wall temperature
#k_W, k_M, x_W_t, x_W_S estimation
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv') #For control hive
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_no_treat.csv') #For treated hive -> only non-treated days
data = data[:10*24] #first seven days of data

In [ ]:
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W, x_W, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W + x_W*k_W*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_W, x_W, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_W, x_W, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_M = ps['k_M'].value #lumped thermal conductivity hive core
    x_W = ps['x_W'].value
    k_W = ps['k_W'].value
    model = []
    for i,j in data_group:
        environment_temp = j['Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_W, x_W, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Temp_Easy']).ravel()

params = Parameters()
params.add('k_M', value = 1, min = 0, max = 1)
params.add('x_W', value = 0.005, min = 0.000001, max = np.inf)
params.add('k_W', value = 1, min = 0.000001, max = 1)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Temp_Easy'] + result.residual[:len(data['Temp_Easy'])].reshape(data['Temp_Easy'].shape)
print('k_M', result.params['k_M'].value)
print('x_W', result.params['x_W'].value)
print('k_W', result.params['k_W'].value)

In [ ]:
#test the value for entire dataset
import scipy.integrate as scint
data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv')
def bee_eq(y, t, theta_E, solar_rad, theta):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W*5 + 5*x_W*k_W*solar_rad - 5*k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

k_M = result.params['k_M'].value #lumped thermal conductivity hive core
k_W = result.params['k_W'].value #lumped thermal conductivity top wall
x_W = result.params['x_W'].value #0.012482890342958908 #lumped heat transfer coeff top wall  

t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
a = [1]*len(data)
temp_box_recons = run_bee_eq(t,a, data['Temp_Box']+273.15, data['Sol Rad'], data['Temp']+273.15)

In [ ]:
import pandas as pd
#data = pd.read_csv('C:\Bee Temp Data\C265_w_Env.csv') #For control hive
data = pd.read_csv('C:\Bee Temp Data\C294_w_Env_no_treat.csv') #For treated hive -> only non-treated days
data = data[:10*24] #first seven days of data

In [ ]:
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W, x_W, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W + x_W*k_W*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_W, x_W, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_W, x_W, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_M = ps['k_M'].value #lumped thermal conductivity hive core
    x_W = ps['x_W'].value
    k_W = 0.29 #ps['k_W'].value
    model = []
    for i,j in data_group:
        environment_temp = j['Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_W, x_W, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Temp_Easy']).ravel()

params = Parameters()
params.add('k_M', value = 1, min = 0, max = np.inf)
params.add('x_W', value = 0.005, min = 0.000001, max = np.inf)
#params.add('k_W', value = 1, min = 0.000001, max = 1)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Temp_Easy'] + result.residual[:len(data['Temp_Easy'])].reshape(data['Temp_Easy'].shape)
print('k_M', result.params['k_M'].value)
print('x_W', result.params['x_W'].value)
#print('k_W', result.params['k_W'].value)

In [ ]:
import pandas as pd
#data = pd.read_csv('C:\Bee Temp Data\C284_w_Env.csv') #For control hive
data = pd.read_csv('C:\Bee Temp Data\C294_w_Env_no_treat.csv') #For treated hive -> only non-treated days
data = data[:10*24] #first seven days of data

In [ ]:
import numpy as np
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import matplotlib.pyplot as plt

def bee_eq(y, t, theta_E, solar_rad, theta, k_W, x_W, k_M):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W + x_W*k_W*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, hive_temp, k_W, x_W, k_M):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, hive_temp, k_W, x_W, k_M), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

def residual(ps, ts, data):
    data_group = data.groupby('Date')
    k_M = ps['k_M'].value #lumped thermal conductivity hive core
    x_W = 0
    k_W = 0.15#ps['k_W'].value
    model = []
    for i,j in data_group:
        environment_temp = j['Air Temp'].values 
        solar_radiation = j['Sol Rad'].values
        hive_temp = j['Temp'].values
        
        t_max = len(environment_temp)-1
        t = np.linspace(0,t_max, num = t_max+1)
        a = [1]*len(environment_temp)
        fitted_temp = run_bee_eq(t, a, environment_temp, solar_radiation, hive_temp, k_W, x_W, k_M)
        
        model = np.concatenate((model, fitted_temp))
    
    return (model - data['Temp_Easy']).ravel()

params = Parameters()
params.add('k_M', value = 1, min = 0, max = np.inf)
#params.add('k_W', value = 1, min = 0.000001, max = 1)
t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
#result = minimize(residual, params, args=(t, data),method='brute',nan_policy='omit', Ns = 20)
result = minimize(residual, params, args=(t, data),method='nelder',nan_policy='omit')
sig_min_front = data['Temp_Easy'] + result.residual[:len(data['Temp_Easy'])].reshape(data['Temp_Easy'].shape)
print('k_M', result.params['k_M'].value)
#print('k_W', result.params['k_W'].value)

In [ ]:
#test the value for entire dataset
import scipy.integrate as scint
data = pd.read_csv('C:\Bee Temp Data\C284_w_Env.csv')
def bee_eq(y, t, theta_E, solar_rad, theta):
    theta_W =  y
    dydt = (theta_E - theta_W)*k_W + x_W*k_W*solar_rad - k_M*(theta_W - theta)
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

k_M = 1 #lumped thermal conductivity hive core
k_W = 1 #lumped thermal conductivity top wall
x_W = 0 #0.012482890342958908 #lumped heat transfer coeff top wall  

t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
a = [1]*len(data)
temp_box_recons = run_bee_eq(t,a, data['Air Temp']+273.15, data['Sol Rad'], data['Temp']+273.15)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp_Easy'], color = 'blue', label = 'Recorded Peripheral Temperature')
plt.plot(temp_box_recons-273.15, color = 'red', label = 'Reconstructed Peripheral Temperature (no ice)')
plt.legend()

In [ ]:
for i in temp_box_recons:
    print(i-273.15)

In [ ]:
import scipy.integrate as scint
import pandas as pd
import numpy as np
data = pd.read_csv('C:\Bee Temp Data\C294_w_Env.csv')
def bee_eq(y, t, theta_E, solar_rad, theta):
    theta_W =  y
    dydt = -(theta_W - theta)*k_M + (theta_E - theta_W)*k_W_s + c_W_s*solar_rad #-4*(theta_W - theta)*k_M + 4*(theta_E - theta_W)*k_W + 4*c_W*solar_rad #+ M
    return dydt

def run_bee_eq(t, a, theta_E, solar_rad, theta):
    sol = scint.odeint(bee_eq, a, t, args=(theta_E, solar_rad, theta), col_deriv = True, rtol = 10e-3, atol = 10e-3) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t 

k_M = 1 # 0.84
k_W_s =  1 #0.6699726312530309 #thermal conductivity of top surface (wood + aluminum plate) -> per hourc
c_W_s =  0*k_W_s #0.0021755653867763236*k_W_S #heat absorption coeff of top surface
k_W = k_W_s #0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
c_W =  c_W_s #0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
    

t_max = len(data)-1
t = np.linspace(0,t_max, num = t_max+1)
a = [1]*len(data)
temp_box_recons = run_bee_eq(t,a, data['Temp_Box']+273.15, data['Sol Rad'], data['Temp']+273.15)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,4), dpi = 200)
plt.plot(data['Temp_Easy'], color = 'blue', label = 'Recorded Peripheral Temperature')
plt.plot(temp_box_recons-273.15, color = 'red', label = 'Reconstructed Peripheral Temperature (no ice)')
plt.legend()

In [ ]:
data['Recons_Peri_k_3'] = temp_box_recons-273.15
data.to_csv('C:\Bee Temp Data\C294_w_Env.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\C2610_w_Env_only_treat.csv')

In [ ]:
import numpy as np
# Constants (you must set these)
k_A_W = 0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
k_hive = 0.84 #[1.5540477862285431]*len(data)
dt_seconds = 1

# 1. Compute dT_wall/dt using finite difference
dT_wall_dt = np.gradient(data['Temp_Easy'], dt_seconds)

# 2. Compute P(t)
data_P = k_A_W*(data['Air Temp'] - data['Temp_Easy']) - k_hive*(data['Temp_Easy'] - data['Temp']) - dT_wall_dt + eta_A_W*data['Sol Rad']

In [ ]:
data['P_less_1_84'] = data_P

In [ ]:
data.to_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')

In [ ]:
import numpy as np
# Constants (you must set these)
k_W = 0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = 0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = 0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = 0.0029*k_W #[0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = 0.84 #[1.5540477862285431]*len(data)
dt_seconds = 1

print(data['Temp_Easy'].values)
# 1. Compute dT_wall/dt using finite difference
dT_wall_dt = np.gradient(data['Temp_Easy'], dt_seconds)
print(dT_wall_dt)

# 2. Compute P(t)
data_P =4*k_W*(data['Air Temp'] - data['Temp_Easy']) + k_A_W*(data['Air Temp'] - data['Temp_Easy']) - 5*k_hive*(data['Temp_Easy'] - data['Temp']) - dT_wall_dt + eta_A_W*data['Sol Rad']+ 4*eta_W*data['Sol Rad']
print(data_P)

In [ ]:
data['P_less_1_84_5'] = data_P

In [ ]:
data.to_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')

In [ ]:
data

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W + k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W 
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t

In [ ]:
data = pd.read_csv('C:\Bee Temp Data\C260_w_Env_only_treat.csv')
data.tail()

In [ ]:
h = data['h_P_less_1_6'].values #health factor per day
theta_ideal = data['theta_P_less_1_6'].values + 273.15 #theta_ideal per day
t_max = len(data['Air Temp'])-1
t = np.linspace(0,t_max, num = t_max+1)
environment_temp = data['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
solar_rad = data['Sol Rad'].values #hourly solar radiation -> kelvin
#k_W = data['k_W'].values
#k_A_W = data['k_A_W'].values
#k_hive = data['k_hive'].values
#eta_W = data['eta_W'].values
#eta_A_W = data['eta_A_W'].values
#eta_B_W = data['eta_B_W'].values
k_W = 0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = 0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = 0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = 0.0029*k_W #[0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = 0.84 #[1.5540477862285431]*len(data)

r_c = 139#1011 #ps['w1'].value
r_h = 1215#6204 #ps['w2'].value
        
a = [1]*len(environment_temp)
        
fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                    k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal)



positive_change = data['P_less_1_6'] > 0

plt.figure(figsize = (10,4), dpi = 200)
xlabel = data['DateTime'].values
y = xlabel[range(0,len(xlabel),24)]
legend_properties = {'size': 10}
ax1 = plt.subplot(211)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax1.plot(data['Temp'], label = 'Recorded Core Temp /w Ice', color = 'orange')
ax1.plot(fitted -273.15, label = 'Reconstructed Core Temp w/o Ice', color = 'red') 
ax1.plot(data['Recons_P_less_1_6'], label = 'Reconstructed Core Temp /w Ice', color = 'blue')

ax1.fill_between(range(len(fitted)), fitted - 273.15, data['Recons_P_less_1_6'], where = positive_change, color='m', alpha=0.3, label='Reduced Core Temperature')
ax1.set_xlabel('Date & Time')
ax1.set_ylabel('Temperature($^0C$)')
ax1.legend(loc='lower right', prop=legend_properties, ncol = 2)
ax1.yaxis.set_tick_params(labelsize=12)
ax1.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date']),12))
#ax1.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax1.set_xticks(range(0,len(data['Date'].values),24))
ax1.set_xticklabels(y,rotation=90)
ax1.set_xticks([])

ax2 = plt.subplot(212)
#plt.plot(data['Temp'], label = 'Hive Core Temperature', color = 'blue')
ax2.plot(data['Temp_Box'], label = 'Empty Box Temperature', color = 'orange')
ax2.plot(data['Recons_Peri'], label = 'Reconstructed Peri Temperature w/o ice', color = 'blue')
ax2.plot(data['Temp_Easy'], label = 'Peri Temp /w ice', color = 'red') 

ax2.fill_between(range(len(data['Temp_Box'])), data['Temp_Box'], data['Temp_Easy'], where = positive_change, color='m', alpha=0.3, label='Saved Bee Energy for Cooling')
ax2.set_xlabel('Date & Time')
ax2.set_ylabel('Temperature($^0C$)')
ax2.legend(loc='lower left', prop=legend_properties)
ax2.yaxis.set_tick_params(labelsize=12)
ax2.xaxis.set_tick_params(labelsize=10)
#ax6.set_ylabel("Health Factor,$\it{h}$", size = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date']),12))
ax2.set_xlabel("Date & Time", fontsize = 14, fontweight='bold')
ax2.set_xticks(range(0,len(data['Date'].values),24))
ax2.set_xticklabels(y,rotation=90)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
data = pd.read_csv('C:\Bee Temp Data\C284_w_Env.csv')
#data = pd.read_csv('C:\Bee Temp Data\C260_w_Env.csv')
data.tail()

In [ ]:
def bee_wall_eq(y, t, env_temp, solar, k_W, k_hive, eta_W, hive_temp, P):
    wall_temp = y
    A_W = k_W*(-wall_temp + env_temp) + eta_W*solar + k_hive*(-wall_temp + hive_temp) - P
    dydt = A_W 
    return dydt

def run_bee_wall_eq(t, a, env_temp, solar, k_W, k_hive, eta_W, hive_temp, P):
    sol = scint.odeint(bee_wall_eq, a, t, args=(env_temp, solar, k_W, k_hive, eta_W, hive_temp, P), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t  

In [ ]:
h = data['h_P'].values #health factor per day
theta_ideal = data['theta_P'].values + 273.15 #theta_ideal per day
t_max = len(data['Air Temp'])-1
t = np.linspace(0,t_max, num = t_max+1)
environment_temp = data['Air Temp'].values + 273.15 #hourly environment temperature -> kelvin
solar_rad = data['Sol Rad'].values #hourly solar radiation -> kelvin
hive_temp = data['Temp'].values + 273.15

k_W = 0.29 #0.9322636905520281 #[9.974510910778044]*len(data) #thermal conductivity of wood -> per hour
eta_W = 0.014*0.29 #0.0033021413705307534*k_W #[0.003307946071640233*9.974510910778044]*len(data) #heat absorption coeff of wood
k_A_W = 0.29 #0.6699726312530309 #[7.206896551724138]*len(data) #thermal conductivity of top surface (wood + aluminum plate) -> per hour
eta_A_W = 0.014*0.29 #0.0021755653867763236*k_A_W #[0.0021482758620689655*7.206896551724138]*len(data) #heat absorption coeff of top surface
eta_B_W = 0.014*0.29 #0.0029*k_W #[0.0029*9.974510910778044]*len(data) #heat absorption coeff of bottom surface
k_hive = 0.74 #1.07 #0.84 #[1.5540477862285431]*len(data)
P = 0 #data['P'].values
        
a = [1]*len(environment_temp)

fitted_wall = run_bee_wall_eq(t, a, environment_temp, solar_rad, k_W, k_hive, eta_W, hive_temp, P)

data['Recons_Wall'] = fitted_wall - 273.15
data.to_csv('C:\Bee Temp Data\C284_w_Env.csv')